# Rapport

ETAPES:

a)Partie analyse lineaire:

    1-discrétisation de la structure 
    2-création des matrices K  et F
    3-mise en place des conditions aux limites
    4-resolution de K*U=F => obtention de U

b)Partie analyse du flambage 

    1-construction de la matrice G
    2-resolution de l'equation (K-lambda*G)*U=0 grace à la méthode puissance itérée 
    => obtention de lambda
    3-Vérification de notre coefficient lambda

Dans un premier temps, on a créé un modèle de notre structure en la discrétisant.

Nous avons par un cas simple, soit une poutre.
    
Nous avons décomposé une poutre en $n_{e}$ segments. Elle est constituée d'**éléments**, separés par $n$ **noeuds** avec pour coordonnées $\{(x_I,y_I)\}_{I=1}^{n}$.

Nous avons ensuite appliqué cette discrétisation à une structure plus complexe, le portique.

# I-Partie analyse du flambement de structure

## 1) Construction de la matrice G

In [2]:
%matplotlib inline  
from sympy.interactive import printing
printing.init_printing()
from frame import *
import sympy as sp
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg

Après avoir trouvé la **solution fondamentale** du problème, nous avons commencé la résolution du problème de flambement de notre structure.

La résolution de ce problème consiste à trouver le coefficient multiplicateur de charges, $\lambda$.

Pour cela, on doit résoudre le problème :
$$
(K-\lambda G)U = 0
$$

avec $G$ la **matrice de rigidité géométrique**

On construit $G$ à l'aide la formule suivante: 
$$
G^{(e)}_{ij}=-\bar N_0^{(e)}\int_0^LS_{1i}'(s)S_{1j}'(s) ds,\quad i,j=1,\ldots,6
$$

avec $N_0^{(e)}$ la valeur normale de l'effort pour un élément $e$

Nous avons donc tout d'abord récupérer les valeurs des efforts normaux pour chaque élément puis nous les avons mis dans un tableau.

In [3]:
class Frame_Buckling(LinearFrame):

    def N_local_stress(self,element):

        """
        Returns the normal forces of an element.
        """
        Ke= self.K_local()
        Ue= self.U_e_local_coord(element)
        F=Ke*Ue
        N_local = F[3]
        return N_local
    
    def N_local_stress_tot(self):

        """
        Returns the normal force of all elements.
        """
        Ns=[self.N_local_stress(e)for e in range (self.nelements)]
        return Ns
    
    

Nous avons utilisé le principe d'héritage pour créer cette classe. En effet, cette classe dépend de la classe utilisée précédemment, $LinearFrame$

In [ ]:
class Frame_Buckling(LinearFrame):
    def G_local(self):
        """
        Returns the global geometric stiffness matrix
        """
        L = sp.Symbol('L')
        s = sp.Symbol('s')
        S=self.S()
        Ge=sp.Matrix([[sp.integrate(S[1,i_local].diff(s)*S[1,j_local].diff(s),(s,0,L) )for i_local in range(6)] for j_local in range(6)]) 
        return Ge
    
    def G_local_rotated(self):
        """
        Gives the analytical expression the local geometric stiffness matrix in the global coordinate system 
        as a function of the orientation angle alpha
        """
        alpha = sp.Symbol("alpha")
        R = self.rotation_matrix(alpha)
        Ge = R.transpose()*self.G_local()*R
        return Ge
    
    def assemble_G(self):
        """
        Returns the global stiffness matrix
        """
        Ge = self.G_local_rotated()
        G = np.zeros([self.ndof,self.ndof])
        N0=self.N_local_stress_tot()
        for e in range(self.nelements):
            Gen = -N0[e].subs({'EI': self.EI[e], 'ES': self.ES[e], 'L': self.Ls[e], 'alpha': self.angles[e]})*Ge.subs({'EI': self.EI[e], 'ES': self.ES[e], 'L': self.Ls[e], 'alpha': self.angles[e]})
            for i_local in range(6):
                for j_local in range(6):
                    G[self.dof_map(e, i_local),self.dof_map(e, j_local)] += Gen[i_local,j_local]
        return G
    
    def bc_apply_G(self,G,blocked_dof):
        for (dof) in enumerate(blocked_dof): 
            Gbc = G 
            Gbc[dof, :] = 0
            Gbc[:, dof] = 0
            Gbc[dof, dof] = 1
        return Gbc